In [67]:
#load necessary libraries
import numpy as np # use number matrices
import pandas as pd # work with data as tables
import matplotlib.pyplot as plt 
import tensorflow as tf

In [68]:
# Step 1 Load the data
df = pd.read_csv("data.csv")
# Remove the features less important, axis = 1 means remove cols
df = df.drop(["index", "price", "sq_price"], axis = 1)
# Only select top 10 rows
df_10 = df[:10]

In [69]:
# Step 2 Create the labels
df_10.loc[:,("y1")] = [1,1,1,0,0,1,0,1,1,1]

/Users/isaac/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/isaac/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [70]:
#y2 is the negation of y1
#.astype(int) convert True or False to binary
df_10.loc[:,("y2")] = (df_10["y1"] == 0).astype(int)

/Users/isaac/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/isaac/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [71]:
#Step 3 prepare data for tensor flow, convert all data to tensors
#Tensors are generic version of vectors and matrices
#Vectors is a list of numbers (1D)
#matrices is a list of vectors (2D)
#Convert features to input tensors
inputX = df_10.loc[:, ["area", "bathrooms"]].as_matrix()
#Convert labels to tensors
inputY = df_10.loc[:,["y1", "y2"]].as_matrix()

In [72]:
#Step 4 to write out hyperparameters
learning_rate = 0.000001
training_epochs = 2000
disp_steps = 50
n_sample = inputY.size

In [73]:
#Step 5 Create computation graph/neural network
#For feature input tensors, none means any number of examples
#2 means two features
#place holders are gateways for data into our computation graph
x = tf.placeholder(tf.float32, [None, 2])

#Set up weights
#2x2 matrix that we'll keep updating through
#the training process
#variables in tf hold and update parameters
#in memory buffers containing tensors
w_weights = tf.Variable(tf.zeros([2,2]))

#add biases
#similar to y = mx + b, b is biases
b_biases = tf.Variable(tf.zeros([2]))

#multiply input by weights and add biases
y_values = tf.add(tf.matmul(x, w_weights), b_biases)

#apply softmax/sigmod to value 
#softmax is the activation function
y = tf.nn.softmax(y_values)

#feed in a matrix of labels
y_ = tf.placeholder(tf.float32,[None, 2])

In [74]:
#Step 6 is to perform a training step
#Create a cost function, mean square error
cost = tf.reduce_sum(tf.pow(y_ - y, 2)) / (2 * n_sample)
#Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [75]:
#initialize variables and tensorflow sessions
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [65]:
#Step 7 actual training loop
for i in range(training_epochs):
    sess.run(optimizer, feed_dict={x: inputX, y_: inputY})
    if i % disp_steps == 0:
        cc = sess.run(cost, feed_dict={x: inputX, y_: inputY})
        print ("Training Step {a}, cost = {b}".format(a = i, b = cc))

print("Optimization Finished!")
training_cost = sess.run(cost, feed_dict={x: inputX, y_: inputY})
print("Training cost = {c}, Weight = {w}, Biases = {b}".format(c = training_cost, w = sess.run(w_weights), b = sess.run(b_biases)))

Training Step 0, cost = 0.1095372885465622
Training Step 50, cost = 0.10953720659017563
Training Step 100, cost = 0.10953714698553085
Training Step 150, cost = 0.10953707993030548
Training Step 200, cost = 0.1095370203256607
Training Step 250, cost = 0.10953693091869354
Training Step 300, cost = 0.10953687131404877
Training Step 350, cost = 0.1095368042588234
Training Step 400, cost = 0.10953674465417862
Training Step 450, cost = 0.10953667014837265
Training Step 500, cost = 0.10953660309314728
Training Step 550, cost = 0.10953652858734131
Training Step 600, cost = 0.10953645408153534
Training Step 650, cost = 0.10953639447689056
Training Step 700, cost = 0.109536312520504
Training Step 750, cost = 0.10953625291585922
Training Step 800, cost = 0.10953619331121445
Training Step 850, cost = 0.10953612625598907
Training Step 900, cost = 0.1095360517501831
Training Step 950, cost = 0.10953597724437714
Training Step 1000, cost = 0.10953591763973236
Training Step 1050, cost = 0.1095358356833

In [66]:
sess.run(y, feed_dict={x: inputX})

array([[ 0.71123087,  0.28876919],
       [ 0.66498965,  0.33501032],
       [ 0.73654455,  0.26345551],
       [ 0.64717317,  0.3528268 ],
       [ 0.78332442,  0.21667559],
       [ 0.70069939,  0.29930058],
       [ 0.65866619,  0.34133381],
       [ 0.64829433,  0.3517057 ],
       [ 0.64369309,  0.35630691],
       [ 0.65480596,  0.34519401]], dtype=float32)

In [ ]:
#Above results indicate that all houses are good buy
#not true, as the good buy is 7/10
#how to imporve?
#add a hidden layer